# **Creating a Global Friction Surface Using QGIS**

## Intro

This guide will take you through the process of creating a global friction surface for estimating travel times to cities in the 1980s. The goal is to integrate various raster data sources—such as urbanization, shipping lanes, topography, country borders, road networks, and more—into a cohesive friction surface that models the travel time to the nearest urban center from any given location.

To create the desired friction surface, we identified nine key features that have the greatest impact on travel time. These include Border Crossing Time, Ports, Roads, Railways, Navigable Rivers, Shipping Lanes, Urbanization, Land Cover, and Elevation. After pinpointing these features, we gathered raster data corresponding to the necessary information and transformed each into a distinct friction layer. Below is a table listing all the datasets used in this guide along with their respective uses.


| Use                     | Description                                      | Link                                                                 |
|-------------------------|--------------------------------------------------|----------------------------------------------------------------------|
| Border Crossing Time    | Shapefile of all country borders                 | [ne_50m_admin_0_boundary_lines_land](https://github.com/nvkelso/natural-earth-vector/blob/master/50m_cultural/ne_50m_admin_0_boundary_lines_land.shp) |
| Border Crossing Time    | Shapefile of all Continents                      | [Continent_raster](Continent_raster.tif)                                         |
| World Map               | World Map                                        | [international.ipums.org](https://international.ipums.org/international/gis.shtml)      |
| Port Data               | Port Data                                        | [World Port Index](https://fgmod.nga.mil/apps/WPI-Viewer/)                       |
| Road Network            | Road Network                                     | [hgl.harvard.edu](https://hgl.harvard.edu/catalog/harvard-dcw-rd-line)          |
| Railway Network         | Railway Network                                  | [hgl.harvard.edu](https://hgl.harvard.edu/catalog/harvard-dcw-rr-line)          |
| Navigable Rivers        | Navigable Rivers                                 | [gaia.edu](http://gaia.geosci.unc.edu/rivers/)                           |
| Major Water Bodies      | Major Water Bodies                               | [worldwildlife.org](https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database) |
| Shipping Lanes          | Shipping Lanes                                   | [Stanford Digital Repository](https://purl.stanford.edu/zv735jp7944)                        |
| National Borders (Current) | National Borders (Current)                   | [natural-earth-vector](https://github.com/nvkelso/natural-earth-vector/blob/master/50m_cultural/ne_50m_admin_0_boundary_lines_land.shp) |
| Degree of Urbanization  | Degree of Urbanization                           | [jeodpp.jrc.ec.europa.eu](https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_SMOD_GLOBE_R2023A/GHS_SMOD_E1980_GLOBE_R2023A_54009_1000/V1-0/) |
| Land Cover              | Land Cover                                       | [jeodpp.jrc.ec.europa.eu](https://forobs.jrc.ec.europa.eu/glc2000/data)                 |
| Elevation               | Elevation                                        | [usgs.gov](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-global-30-arc-second-elevation-gtopo30?qt-science_center_objects=0#qt-science_center_objects) |



---

# **Part 1: Upload World Map**

### Step 1: Upload the World Map shapefile 
- **Objective**: Load the shapefile for the World Map  into QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.

### Step 2: Reproject the World Map Vector
- **Objective**: Reproject the World Map vector layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Reproject Layer'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the World Map vector.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
Click ‘Run’ 


---
---

# **Part 2: Make 1km x 1km grid**


### Step 1: Create 1km x 1km grid
- **Objective**: Set the standard grid that all raster resolutions will be based on.
  - QGIS Toolbox: Search for “Create Grid”.
  - Grid Type: Rectangle.
  - Grid Extent: World Map.

---

### Code

In [ ]:
processing.run("native:creategrid", {'TYPE':2,'EXTENT':'-180.000000000,180.000000000,-89.999983420,83.633392334 [EPSG:4326]','HSPACING':0.033333,'VSPACING':0.033333,'HOVERLAY':0,'VOVERLAY':0,'CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'OUTPUT':'TEMPORARY_OUTPUT'})

---
---

# **Part 3: Individual Trasformations of the Ratser Data**

# 1. Border Crossing Time Calculation

This guide outlines how to calculate border crossing times by assigning friction weights to borders based on continent-level analysis. We will reproject the continent raster and rasterize the country borders to represent the time it takes to cross borders. This simplified approach helps to account for over 80 countries efficiently.

## Step-by-Step Process

### Step 1: Open Shapefiles
- **Objective**: Load the shapefile for all country borders and all continents into QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.
  - Similarly, load the shapefile containing the continents layer.

### Step 2: Reproject the Continent Raster
- **Objective**: Reproject the continent raster layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Warp (Reproject)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the continent raster.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.

### Step 3: Change the Resolution
- **Objective**: have the borders in one kilometer pixel resolution
  - Open the **r.resample** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the reprojected continent raster.
  - Set the **GRASS GIS 7** region cellsize to 0.008333

### Step 4: Convert Country Borders to Raster
- **Objective**: Convert the country borders vector layer (line geometry) into a raster format.
  - Open the **'Rasterize (Vector to Raster)'** tool from the Processing Toolbox.
  - In the **Input Layer**, select the country borders layer.
  - Set both **Width/Horizontal resolution** and **Height/Vertical resolution** to **0.00833** to create a raster with appropriate resolution.
  - This step converts vector country boundaries into a grid format, where each pixel will represent a specific area.

### Step 5: Fill Null Values in Raster
- **Objective**: Ensure all NULL values in the raster are filled appropriately.
  - Open the **'Fill Data'** tool from the Processing Toolbox.
  - In the **Input Layer** select the newly made raster border file
  - Set the **Fill Value** attribute to **0** 

### Step 6: Modify Border Raster with a Distinct Values
- **Objective**: Assign a new value to the modified border raster to help with later calculations.
  - In the **Input Layer** select the newly filled raster border file
  - Open the **'Raster Calculator'** from the Processing Toolbox.
  - In the calculator, use the formula:
    ``` 
    ("guide_border_fill@1" = 1) * 0.5
    ```
    - This assigns a friction weight (0.5) to the areas of the border raster that have value 1, representing the country borders.
  - Save the output as **'halved_modified_borders'** to preserve this modified raster.

### Step 7: Calculate the Difference Between Continent and Border Rasters
- **Objective**: Subtract the modified border raster from the continent raster to identify differences, so that each border has a unqiue value cooresponding to its continent.
  - Open the **'Raster Calculator'** again.
  - Use the formula:
    ``` 
    "Continent_raster@1" - "guide_border_fill@1"
    ```
  - This operation will highlight the differences between the continent raster and the border data, which can then be used to calculate border crossing times.
  - Save the output as **'continent_border_diff'**.

### Step 8: Assign Friction Weights Based on Border Differences
- **Objective**: Assign specific friction weights based on the values in the **'continent_border_diff'** raster.
  - Open the **'Raster Calculator'** once more.
  - Use the following formula to assign specific friction values based on the differences between continents and borders:
    C 
    ("continent_border_diff@1" = 0.5) * 0.115 (.0087) (80) + 
    ("continent_border_diff@1" = 1.5) * 0.075 (.013.33) (55)+ 
    ("continent_border_diff@1" = 2.5) * 0.075 (.013.33) (55)+ 
    ("continent_border_diff@1" = 3.5) * 0.06 (.016.67) (45)+ 
    ("continent_border_diff@1" = 4.5) * 0.075 (013.33) (55)+ 
    ("continent_border_diff@1" = 5.5) * 0.055 (18.19)+ (40)
    ("continent_border_diff@1" = 6.5) * 0.075 (13.33) (55)+ 
    ("continent_border_diff@1" = 7.5) * 0.035 (28.57) (30)
    ```
    - This formula assigns different friction weights based on the values found in **'continent_border_diff'**. The weights (e.g., 0.115, 0.075) represent time values (minutes per meter) associated with different levels of border difficulty.


| Continent       | Friction Weight | Description | Explanation |
|-----------------|-----------------|-------------|-------------|
| Africa          | 80 Min/km | ["Land Transport for Export: The Effects of Cost, Time, and Uncertainty in Sub-Saharan Africa"]() and ["Mapping travel time to assess accessibility in West Africa: The role of borders, checkpoints and road conditions"]() | High friction due to poor infrastructure, numerous checkpoints, and complex border procedures. |
| Asia            | 55 Min/km| [ITF Southeast Asia Transport Outlook](https://www.itf-oecd.org/sites/default/files/docs/itf-southeast-asia-outlook.pdf), [TRADE IMPACT OF REDUCING TIME AND COSTS AT BORDERS IN THE CENTRAL ASIA REGIONAL ECONOMIC COOPERATION REGION](https://www.adb.org/sites/default/files/publication/575606/adbi-wp1106.pdf) | Moderate friction due to varying infrastructure quality and border management efficiency. |
| Europe          | 45 Min/km| [Abstract from latest IRU Working Papers on Road Border Crossing Improvements](https://www.iru.org/sites/default/files/2016-01/en-krauss-bwto.pdf) | Lower friction due to well-developed infrastructure and streamlined border procedures. |
| North America   | 55 Min/km| [horizonsunlimited.com](https://www.horizonsunlimited.com/tripplan/borders/central-america)| Moderate friction due to varying infrastructure quality and border management efficiency. |
| Oceania         | 40 Min/km| [Self-reported wait times for the PNG and Indonesian land border](https://www.tripadvisor.com/Attraction_Review-g3411772-d5562474-Reviews-The_Border_Between_PNG_and_West_Papua-Vanimo_Momase_Region.html) | Lower friction due to less congested borders and simpler procedures. |
| South America   | 55 Min/km| [advtracks.online](https://advtracks.online/blog/post/19_border-crossings-south-america) | Moderate friction due to varying infrastructure quality and border management efficiency. |

### Final Output
By following these steps, you’ll convert your country border and continent shapefiles into a raster format, modify the raster with specific friction values, and calculate border crossing times based on these friction values. The final output will provide you with a simplified, continent-level friction map that helps estimate border crossing times across the globe.

---


<div style="text-align: center;">
    <p style="text-align: center;">Before Borders Processing</p>
    <img src="before_borders.png" alt="Before Borders Processing" width="800"/>  <br> 
    <img src="down-chevron.png" alt="Down Arrow" width="110"/>  <br> 
    <p style="text-align: center;">After Borders Processing</p>
    <img src="borders_end.png" alt="After Borders Processing" width="800"/>
</div>


### Code

In [ ]:

# Code to turn the country borders from a vector to a raster
processing.run("gdal:rasterize", {'INPUT':'[input file path]','FIELD':'','BURN':1,'USE_Z':False,'UNITS':1,'WIDTH':0.008333,'HEIGHT':0.008333,'EXTENT':'-115.967578128,139.498377155,-56.294072926,93.400704263 [EPSG:4326]','NODATA':0,'OPTIONS':'','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'C:/Users/samja/Downloads/gudie_raster_borders.tif'})

# Code to rescale the raster to the same size as the continent raster
processing.run("grass7:r.resample", {'input':'[input file path]','output':'C:/Users/samja/Documents/borders_resampled.tif','GRASS_REGION_PARAMETER':None,'GRASS_REGION_CELLSIZE_PARAMETER':0.008333333333333333,'GRASS_RASTER_FORMAT_OPT':'','GRASS_RASTER_FORMAT_META':''})

# Code to fill in the nodata values
processing.run("native:fillnodata", {'INPUT':'[input file path]','BAND':1,'FILL_VALUE':0,'OUTPUT':'C:/Users/samja/Downloads/guide_border_fill.tif'})

# Code to scale down border value
processing.run("native:rastercalc", {'LAYERS':['[input file path]'],'EXPRESSION':'("guide_border_fill@1" = 1) * 0.5','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

# Code subtract borders from continent raster so that each border has a unqiue value based on continet
processing.run("native:rastercalc", {'LAYERS':['[input file path]','[input file path]'],'EXPRESSION':'"Continent_raster@1"-"guide_border_fill@1"','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

# Code to assign friction values to the borders based on the continent
processing.run("native:rastercalc", {'LAYERS':['[input file path]'],'EXPRESSION':'("continent_border_diff" = 0.5) * 0.115 + ("continent_border_diff" = 1.5) * 0.075 + ("continent_border_diff" = 2.5) * 0.075 + ("continent_border_diff" = 3.5) * 0.06 + ("continent_border_diff" = 4.5) * 0.075 + ("continent_border_diff" = 5.5) * 0.055 + ("continent_border_diff" = 6.5) * 0.075 +("continent_border_diff" = 7.5) * 0.035','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

---

## 2. Shipping lane friction calculation

### Step 1: Open Shapefiles
- **Objective**: Load the shapefile for shipping lanes in QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing shipping lanes.

### Step 2: Reproject the Shipping Lane Raster
- **Objective**: Reproject the shipping lane raster layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Warp (Reproject)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the shipping lane raster.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
  - Run the following code:
    ```
    processing.run("gdal:warpreproject", {'INPUT':'/vsizip/C:/Users/samja/Downloads/raw_2013_shipping_mol_20150714094045.zip/shipping.tif','SOURCE_CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'','DATA_TYPE':0,'TARGET_EXTENT':None,'TARGET_EXTENT_CRS':None,'MULTITHREADING':False,'EXTRA':'','OUTPUT':'C:/Users/samja/OneDrive/Documents/shipping_reprojection.tif'})
    ```

### Step 3: Binarize the Shipping Lane Raster
- **Objective**: Binarize the shipping lanes so that only shipping lanes that receive above a threshold amount of traffic show up.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the reprojected shipping lane raster.
  - Enter the following expression:
    ```
    ("shipping@1" < 0.3) * 1 + ("shipping@1" >= 0.3) * 0
    ```

### Step 4: Assign a Friction Value
- **Objective**: Assign a value to the shipping lanes that represents the cost to travel.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the binarized shipping lane raster.
  - Enter the following expression:
    ```
    ("shipping@1" <= 0) * 0 + ("shipping@1" >= 1) * 0.02
    ```




---

<div style="text-align: center;">
    <p style="text-align: center;">Before Shipping Lane Processing</p>
    <img src="Pre_Shipping_Lanes.png" alt="Image Description" width="1000"/> <br> 
    <img src="down-chevron.png" alt="Image Description" width="110"/>  <br>
    <p style="text-align: center;">After Shipping Lane Processing</p>
    <img src="Reduced_Shipping.png" alt="Image Description" width="1000"/>
</div>

### Code

In [ ]:
# Code to reproject
processing.run("gdal:warpreproject", {'INPUT':'[input file path]','SOURCE_CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'','DATA_TYPE':0,'TARGET_EXTENT':None,'TARGET_EXTENT_CRS':None,'MULTITHREADING':False,'EXTRA':'','OUTPUT':'C:/Users/samja/OneDrive/Documents/shipping_reprojection.tif'})
# Code to binarize shipping
processing.run("native:rastercalc", {'LAYERS':['[input file path]'],'EXPRESSION':'"shipping@1" >= 0.4 * 1 +"shipping@1" < 0.4 * 1','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'C:/Users/samja/Downloads/Shipping_binary.tif'})
# Code to assign friction values to shipping
processing.run("native:rastercalc", {'LAYERS':['[input file path]'],'EXPRESSION':'"shipping@1" >= 0.4 * 1 +"shipping@1" < 0.4 * 1','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'C:/Users/samja/Downloads/Shipping_binary.tif'})

---

## 3. Urban Zone friction calculation

### Step 1: Open Shapefiles
- **Objective**: Load the shapefile for shipping lanes in QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing a raster with urban classifications.

### Step 2: Reproject the Urban Classification Raster
- **Objective**: Reproject the shipping lane raster layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Warp (Reproject)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the urban classifications raster.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
  - Run the following code:
    ```
    processing.run("gdal:warpreproject", {'INPUT':'/vsizip/C:/Users/samja/Downloads/raw_2013_shipping_mol_20150714094045.zip/shipping.tif','SOURCE_CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'','DATA_TYPE':0,'TARGET_EXTENT':None,'TARGET_EXTENT_CRS':None,'MULTITHREADING':False,'EXTRA':'','OUTPUT':'C:/Users/samja/OneDrive/Documents/shipping_reprojection.tif'})
    ```

### Step 3: Binarize the Cities from other Urban Classification
- **Objective**: Binarize the urban classification so that only the most dense urbana zones show up.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the reprojected urban classification raster.
  - Enter the following expression:
    ```
    ("Urban_Classification@1" >= 6) * 1 + ("Urban_Classification@1" < 6) * 0
    ```

### Step 4: Assign a Friction Value
- **Objective**: Assign a value to the shipping lanes that represents the cost to travel.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the binarized shipping lane raster.
  - Enter the following expression:
    ```
    ("Urban_Classification@1" <= 0) * 0 + ("Urban_Classification@1" >= 1) * 0.1
    ```




<div style="text-align: center;">
    <p style="text-align: center;">Urban Zones</p>
    <img src="Urban_Zones_PNG.png" alt="Urban Zones" width="800"/> <br> 
    <img src="down-chevron.png" alt="Down Arrow" width="110"/>  <br> 
    <p style="text-align: center;">Only Urban Areas</p>
    <img src="Only_Urban.png" alt="Only Urban Areas" width="800"/>
</div>

---

## 4. Upload and Reproject Road Network, Railway, and Navigable Rivers


### Goal: Upload and Reproject Port Data, Road Network, Railway, Navigable Rivers, and Major Water Bodies

### Step 1: Open Shapefiles
- **Objective**: Load the shapefiles into QGIS.
    - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the above datasets.

### Step 2: Reproject the Port Data, Road Network, Railway, Navigable Rivers, and Major Water Bodies
- **Objective**: Reproject the Port Data, Road Network, Railway, Navigable Rivers, and Major Water Bodies vector layers into the WGS 84 projection (EPSG:4326).
    - Open the **'Reproject Layer'** tool from the Processing Toolbox.
    - In the **Input Layer** field, select the layer you intend to reproject.
    - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
    - Click ‘Run’.
    - Rename the reprojected layer to “Filename_Rep” to indicate that it has been reprojected.
    - Repeat for all five layers.

Example for European Rivers:
```python
processing.run("native:reprojectlayer", {'INPUT':'/Users/hollyzhang/Documents/QGIS Downloads/europe/eurivs.shp','TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'CONVERT_CURVED_GEOMETRIES':False,'OPERATION':'+proj=noop','OUTPUT':'TEMPORARY_OUTPUT'})
```

### Step 3: Rasterize Road Network, Railway, Navigable Rivers, and Major Water Bodies
- **Objective**: Convert the vector layers into raster format for further analysis.
    - Find the “Vector to Raster” function in the QGIS processing toolbox.
    - Select the reprojected layer you want to rasterize as the input layer.
    - Set “Output raster size units” to “Georeferenced units”.
    - Set “Width/Horizontal Resolution” and “Height/Vertical Resolution” to 0.033333.
    - Set “Output Extent” to your World Map layer.
    - Click “Run”.
    - Rename the rasterized layer to indicate that it has been rasterized.

### Step 4: Assign Travel Speeds to Each Layer
- **Objective**: Assign travel speeds to each raster layer to represent the cost of travel.
    - Open “Raster Calculator” from the processing toolbox.
    - Select the appropriate rasterized input layer.
    - Paste the corresponding expression into “Expression”:

| Layer                | Travel Speed               |
|----------------------|----------------------------|
| Road Network         | Motorways, 120km/hr = 0.5 min per km; Major roads, 60km/hr = 1 min per km; Tracks, 10km/hr = 6 min per km |
| Railway              | 40 km/hr = 1.5 min per km  |
| Navigable Rivers     | 20km/hr = 3 min per km     |
| Major Water Bodies   | 20km/hr = 3 min per km     |

Expressions:
    - Road Network: `("Road_Network_Rasterized" = 1) * 1`
    - Railway: `("Railway_Rasterized" = 1) * 40`
    - Navigable Rivers: `("Navigable_Rivers_Rasterized" = 1) * 20`
    - Major Water Bodies: `("Major_Water_Bodies_Rasterized" = 1) * 20`

---
---

# **Part 4: Combining all processed rasters into one friction layer**

### Step 1: Use the Merge Tool
- **Objective**: Combine multiple raster layers into a single raster layer using the minimum value at each pixel location.
    - Open QGIS, then use the **'Raster Calculator'** tool from the Processing Toolbox.
    - In the **Input Layers** field, select two of the raster layers you want to merge.
        - Border Crossing Time
        - Shipping Lanes
        - Urban Zones
        - Road Network
        - Railway Network
        - Navigable Rivers
        - Major Water Bodies
    - In the calculate enter MIN(Raster_layer1, Raster_layer2)
    - Click **Run** to execute the merge process.
    - Repeat this process by merging an additional layer to the previous output of the Raster Calculator.



### Code

In [ ]:
processing.run("native:rastercalc", {'LAYERS':['[input layer one]','[input layer one]'],'EXPRESSION':'MIN ("Shipping_binary_4@1","urban_presentation@1")','EXTENT':None,'CELL_SIZE':None,'CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'OUTPUT':'TEMPORARY_OUTPUT'})

---
---

# **Part 5: Creating Centroids to Represent Urban Centers**

### Step 1: Open Urban Zone Shapefiles
- Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing discrete urbanization categories.

### Step 2: Binarize Urban Zones
- **Objective**: Extract urban centers from population density data.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the urbanization raster.

### Step 3: Vectorize the Raster
- **Objective**: Convert the binarized urban zones into vector format for further processing.
  - Open the **'Polygonize (Raster to Vector)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the binarized urbanization raster.

### Step 4: Calculate Centroids
- **Objective**: Determine the centroids of the urban zones to represent urban centers.
  - Open the **'Centroids'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the vectorized urban areas.
  - Click **Run** to generate the centroids representing urban centers.


----

<div style="text-align: center;">
    <p style="text-align: center;">Urban Area Before Centroid Calculation</p>
    <img src="Only_Urban.png" alt="Only Urban Areas" width="800"/> <br> 
    <img src="down-chevron.png" alt="Down Arrow" width="110"/>  <br>
    <p style="text-align: center;">Centroids After Centroid Calculation</p>
    <img src="Urban_centroid_black.png" alt="Urban Centroids" width="800"/>
</div>

### Code

In [ ]:
# running the vectorize function
processing.run("gdal:polygonize", {'INPUT':'[input file path]','BAND':1,'FIELD':'DN','EIGHT_CONNECTEDNESS':False,'EXTRA':'','OUTPUT':'C:/Users/samja/Documents/only_city_vectorize.gpkg'})
# running the centroid function
processing.run("native:centroids", {'INPUT':'[input file path]','ALL_PARTS':True,'OUTPUT':'C:/Users/samja/OneDrive/Documents/only_cities_centroids.gpkg'})

---
---

# **Part 5.5: Reduce City Centroids (Optional)**

**Note: These steps explain how to reduce the number of city centroids if you don't have access to a powerful computer.**

### Step 1: Calculate the Area of Each Urban Zone
- **Objective**: Calculate the area of each urban zone vector.
  - Open the **Field Calculator** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the vectorized binary urbanization raster.
  - Under **Field Name**, enter `area`.
  - In the expression section, write `$area`.
  - Click **Run**.

### Step 2: Eliminate Smaller Urban Zones
- **Objective**: Remove less significant urban areas by filtering out smaller zones.
  - Right-click on the output layer from the last step and select **Open Attribute Table**.
  - Click on **Select features using expression**.
  - Enter `$area < {your_threshold}`.
  - Right-click again on the layer and select **Toggle Editing Mode**.
  - Delete all selected features.

### Step 3: Calculate Centroids
- **Objective**: Determine the centroids of the urban zones to represent urban centers.
  - Open the **'Centroids'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the filtered vectorized urban areas.
  - Click **Run** to generate the centroids representing urban centers.


<div style="text-align: center;">
    <p style="text-align: center;">Urban Centroids Before Reduction</p>
    <img src="Urban_centroid_black.png" alt="Urban Centroids Before Reduction" width="800"/> <br> 
    <img src="down-chevron.png" alt="Down Arrow" width="110"/>  <br> 
    <p style="text-align: center;">Urban Centroids After Reduction</p>
    <img src="urban_centroids_reduced.png" alt="Urban Centroids After Reduction" width="800"/>
</div>

### Code

In [ ]:
# step 1
processing.run("native:fieldcalculator", {'INPUT':'[input file path]','FIELD_NAME':'area','FIELD_TYPE':0,'FIELD_LENGTH':0,'FIELD_PRECISION':0,'FORMULA':' $area ','OUTPUT':'TEMPORARY_OUTPUT'})
# step 3
processing.run("native:centroids", {'INPUT':'memory://Polygon?crs=EPSG:4326&field=fid:long(0,0)&field=DN:integer(0,0)&field=area:double(0,0)&uid={1704d9c2-409f-4808-9d02-1c47d9ffdb80}','ALL_PARTS':True,'OUTPUT':'[output file path]'})

# **Part 6: Creating the Cost Distance Surface**

###  Create the Cost Distance Surface
- **Objective**: Generate a cost distance surface using the combined friction layer and urban centroids.
  - Install the r.grass extension.
  - Open the **'r.cost'** tool from the Processing Toolbox.
  - Enter the combined friction layer under the **'Unit Cost Layer'** input.
  - Select the centroids as start points.
  - Run the tool.

<div style="text-align: center;">
    <p style="text-align: center;">Finished Friction Surface</p>
    <img src="finished_friction.png" alt="Finished Friction Surface" width="800"/> 
    <br>
    <img src="—Pngtree—plus vector icon_4015245.png" alt="Plus Icon" width="150" style="display: block; margin-left: auto; margin-right: auto;"/>  
    <br>
    <p style="text-align: center;">Reduced City Centroids<p>
    <img src="reduced_cities.png" alt="Reduced City Centroids" width="800"/> 
    <br> 
    <img src="down-chevron.png" alt="Down Arrow" width="150" style="display: block; margin-left: auto; margin-right: auto;"/>  
    <br>
    <p style="text-align: center;">Cost Distance Surface</p>
    <img src="Cost_map.png" alt="Cost Distance Surface" width="800"/>
</div>


### Code

In [ ]:
# running the cost distance function
processing.run("grass7:r.cost", {'input':'[friction surface file path]','start_coordinates':None,'stop_coordinates':None,'-k':False,'-n':True,'start_points':'[urban centroid file path]','stop_points':None,'start_raster':None,'max_cost':None,'null_cost':None,'memory':5000,'output':'C:/Users/samja/Documents/cumulat_cost.tif','nearest':'C:/Users/samja/Documents/alloca_map.tif','outdir':'C:/Users/samja/Documents/moveme_dir.tif','GRASS_REGION_PARAMETER':None,'GRASS_REGION_CELLSIZE_PARAMETER':0,'GRASS_RASTER_FORMAT_OPT':'','GRASS_RASTER_FORMAT_META':'','GRASS_SNAP_TOLERANCE_PARAMETER':-1,'GRASS_MIN_AREA_PARAMETER':0.0001})

---
---
---